<a href="https://colab.research.google.com/github/balakumar-dataengineer/testrepo/blob/master/Pyspark_Explode()_and_Split()_and_Array()_and_Arrany_contains()_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('Pyspark') \
                          .master('local') \
                          .getOrCreate()

print(spark)

In [3]:
data=[(1,'bala','python','pyspark'),(2,'kumar','sql','gcp')]

schema=['id','name','primarySkill','secondarSkill']

df = spark.createDataFrame(data,schema)

df.show()

+---+-----+------------+-------------+
| id| name|primarySkill|secondarSkill|
+---+-----+------------+-------------+
|  1| bala|      python|      pyspark|
|  2|kumar|         sql|          gcp|
+---+-----+------------+-------------+



In [29]:
from pyspark.sql.functions import array, array_contains, explode, split, concat_ws, concat

df1=df.withColumn('skills',array(df.primarySkill,df.secondarSkill)).drop('primarySkill','secondarSkill')

df1.show()
df1.printSchema()

+---+-----+-----------------+
| id| name|           skills|
+---+-----+-----------------+
|  1| bala|[python, pyspark]|
|  2|kumar|       [sql, gcp]|
+---+-----+-----------------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- skills: array (nullable = false)
 |    |-- element: string (containsNull = true)



In [10]:
df2=df1.withColumn('gcpSkill',array_contains(df1.skills,'gcp')) \
       .withColumn('pythonSkill',array_contains(df1.skills,'python'))

df2.show()
df2.printSchema()

+---+-----+-----------------+--------+-----------+
| id| name|           skills|gcpSkill|pythonSkill|
+---+-----+-----------------+--------+-----------+
|  1| bala|[python, pyspark]|   false|       true|
|  2|kumar|       [sql, gcp]|    true|      false|
+---+-----+-----------------+--------+-----------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- skills: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- gcpSkill: boolean (nullable = true)
 |-- pythonSkill: boolean (nullable = true)



In [15]:
df3=df2.withColumn('skills',explode(df2.skills))

df3.show()
df3.printSchema()

+---+-----+-------+--------+-----------+
| id| name| skills|gcpSkill|pythonSkill|
+---+-----+-------+--------+-----------+
|  1| bala| python|   false|       true|
|  1| bala|pyspark|   false|       true|
|  2|kumar|    sql|    true|      false|
|  2|kumar|    gcp|    true|      false|
+---+-----+-------+--------+-----------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- skills: string (nullable = true)
 |-- gcpSkill: boolean (nullable = true)
 |-- pythonSkill: boolean (nullable = true)



In [21]:
df5=df.withColumn('skills',concat_ws(',',df.primarySkill,df.secondarSkill))
df5.show()
df5.printSchema()

+---+-----+------------+-------------+--------------+
| id| name|primarySkill|secondarSkill|        skills|
+---+-----+------------+-------------+--------------+
|  1| bala|      python|      pyspark|python,pyspark|
|  2|kumar|         sql|          gcp|       sql,gcp|
+---+-----+------------+-------------+--------------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- primarySkill: string (nullable = true)
 |-- secondarSkill: string (nullable = true)
 |-- skills: string (nullable = false)



In [31]:
df6=df5.withColumn('skillsarray',split(df5.skills,','))
df6.show()
df6.printSchema()

df6.withColumn('skillsba',concat(df6.primarySkill,df6.secondarSkill)).show()

+---+-----+------------+-------------+--------------+-----------------+
| id| name|primarySkill|secondarSkill|        skills|      skillsarray|
+---+-----+------------+-------------+--------------+-----------------+
|  1| bala|      python|      pyspark|python,pyspark|[python, pyspark]|
|  2|kumar|         sql|          gcp|       sql,gcp|       [sql, gcp]|
+---+-----+------------+-------------+--------------+-----------------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- primarySkill: string (nullable = true)
 |-- secondarSkill: string (nullable = true)
 |-- skills: string (nullable = false)
 |-- skillsarray: array (nullable = false)
 |    |-- element: string (containsNull = false)

+---+-----+------------+-------------+--------------+-----------------+-------------+
| id| name|primarySkill|secondarSkill|        skills|      skillsarray|     skillsba|
+---+-----+------------+-------------+--------------+-----------------+-------------+
|  1| bala|    